In [47]:
# import packages 
import pandas as pd
import plotly.express as px

# import libraries to access geojson file (for animate chorpleth map)
from urllib.request import urlopen
import json

In [48]:
# data from JHU CSSE
data_covid = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-24-2020.csv"

In [49]:
# make dataframe
df_covid_0424 = pd.read_csv(data_covid)

In [50]:
df_covid_0424.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-25 06:30:53,30.295065,-92.414197,129,7,0,122,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-25 06:30:53,37.767072,-75.632346,100,3,0,97,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-25 06:30:53,43.452658,-116.241552,642,14,0,628,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-25 06:30:53,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US"


In [51]:
# keep only US data
df_covid_0424_us = df_covid_0424[df_covid_0424["Country_Region"]== "US"]

In [52]:
# save csv data from us census population
data_population = "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv"

In [53]:
# import county population total data from 2019
df_countypop = pd.read_csv(data_population,encoding='latin1')

In [54]:
# preview data
df_countypop.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,40,3,6,1,0,Alabama,Alabama,4779736,4780125,4785437,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952


In [55]:
# convert state and county columns to strings
df_countypop["STATE"] = df_countypop["STATE"].astype(str)
df_countypop["COUNTY"] = df_countypop["COUNTY"].astype(str)

In [56]:
# fill in preceding zeros to make state 2 digits and county 3 digits
df_countypop["STATE"] = df_countypop["STATE"].str.zfill(2)
df_countypop["COUNTY"] = df_countypop["COUNTY"].str.zfill(3)

In [57]:
# sum state and county FIPS to create a column that contains the full county fips code
df_countypop["FIPS"] = df_countypop["STATE"] + df_countypop["COUNTY"]

In [58]:
# preview data
df_countypop.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019,FIPS
0,40,3,6,01,000,Alabama,Alabama,4779736,4780125,4785437,...,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744,01000
1,50,3,6,01,001,Alabama,Autauga County,54571,54597,54773,...,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062,01001
2,50,3,6,01,003,Alabama,Baldwin County,182265,182265,183112,...,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567,01003
3,50,3,6,01,005,Alabama,Barbour County,27457,27455,27327,...,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664,01005
4,50,3,6,01,007,Alabama,Bibb County,22915,22915,22870,...,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952,01007


In [59]:
# remove null FIPS codes
df_covid_0424_us = df_covid_0424_us[df_covid_0424_us["FIPS"].notnull()]

In [60]:
# convert JHU df FIPS to integer
df_covid_0424_us["FIPS"] = df_covid_0424_us["FIPS"].astype(int)
df_covid_0424_us["FIPS"] = df_covid_0424_us["FIPS"].astype(str)

In [61]:
# fill in preceding zeros to make FIPS five digits
df_covid_0424_us["FIPS"] = df_covid_0424_us["FIPS"].str.zfill(5)

In [62]:
# preview data
df_covid_0424_us.tail()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
2882,99999,NaN,Grand Princess,US,2020-04-25 06:30:53,NaN,NaN,103,3,0,100,"Grand Princess, US"
2885,00066,NaN,Guam,US,2020-04-25 06:30:53,13.4443,144.7937,141,5,0,136,"Guam, US"
2911,00069,NaN,Northern Mariana Islands,US,2020-04-25 06:30:53,15.0979,145.6739,14,2,0,12,"Northern Mariana Islands, US"
2915,00072,NaN,Puerto Rico,US,2020-04-25 06:30:53,18.2208,-66.5901,1276,77,0,1199,"Puerto Rico, US"
2934,00078,NaN,Virgin Islands,US,2020-04-25 06:30:53,18.3358,-64.8963,54,3,0,51,"Virgin Islands, US"


In [63]:
# merge 2019 population data onto covid-19 df
df_covid_0424_us = pd.merge(df_covid_0424_us,
                           df_countypop[["FIPS","POPESTIMATE2019"]],
                           on = "FIPS",
                           how = "left")

In [64]:
# preview merged data
df_covid_0424_us.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,POPESTIMATE2019
0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0
1,22001,Acadia,Louisiana,US,2020-04-25 06:30:53,30.295065,-92.414197,129,7,0,122,"Acadia, Louisiana, US",62045.0
2,51001,Accomack,Virginia,US,2020-04-25 06:30:53,37.767072,-75.632346,100,3,0,97,"Accomack, Virginia, US",32316.0
3,16001,Ada,Idaho,US,2020-04-25 06:30:53,43.452658,-116.241552,642,14,0,628,"Ada, Idaho, US",481587.0
4,19001,Adair,Iowa,US,2020-04-25 06:30:53,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US",7152.0


In [65]:
# make column for number of cases per 1000 people and deaths per 1000 people
df_covid_0424_us["cases_per1000"] = (df_covid_0424_us["Confirmed"]/df_covid_0424_us["POPESTIMATE2019"])*1000
df_covid_0424_us["deaths_per1000"] = (df_covid_0424_us["Deaths"]/df_covid_0424_us["POPESTIMATE2019"])*1000

In [66]:
# preview data
df_covid_0424_us.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,POPESTIMATE2019,cases_per1000,deaths_per1000
0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.000000
1,22001,Acadia,Louisiana,US,2020-04-25 06:30:53,30.295065,-92.414197,129,7,0,122,"Acadia, Louisiana, US",62045.0,2.079136,0.112821
2,51001,Accomack,Virginia,US,2020-04-25 06:30:53,37.767072,-75.632346,100,3,0,97,"Accomack, Virginia, US",32316.0,3.094442,0.092833
3,16001,Ada,Idaho,US,2020-04-25 06:30:53,43.452658,-116.241552,642,14,0,628,"Ada, Idaho, US",481587.0,1.333092,0.029071
4,19001,Adair,Iowa,US,2020-04-25 06:30:53,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US",7152.0,0.139821,0.000000


In [68]:
# bubble map of number of cases
bubble_map = px.scatter_mapbox(df_covid_0424_us, 
                               lat = "Lat",
                               lon = "Long_",
                               size = "Confirmed",
                               hover_data = ["Confirmed", "Deaths", "cases_per1000", "deaths_per1000"] ,
                               hover_name = "Combined_Key",
                               center = dict(lat=37.0902, 
                                            lon= -95.7129), # center of the US
                               zoom = 2.5,
                               title = "Confirmed COVID-19 Cases in the United States",
                               labels = {"Confirmed": "Cumulative Confirmed Cases",
                                         "Deaths": "Cumulative Confirmed Deaths",
                                         "cases_per1000": "Cases per 1,000 People",
                                         "deaths_per1000": "Deaths per 1,000 People"
                                        } 
                              )

In [69]:
bubble_map

In [70]:
# copy row and append row equal to the number of cases for that county
df_covid_0424_repeat = df_covid_0424_us.reindex(df_covid_0424_us.index.repeat(df_covid_0424_us["Confirmed"])).reset_index()

In [71]:
df_covid_0424_repeat.head(11)

,index,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,POPESTIMATE2019,cases_per1000,deaths_per1000
0,0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.0
1,0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.0
2,0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.0
3,0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.0
4,0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.0
5,0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.0
6,0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.0
7,0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.0
8,0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.0
9,0,45001,Abbeville,South Carolina,US,2020-04-25 06:30:53,34.223334,-82.461707,24,0,0,24,"Abbeville, South Carolina, US",24527.0,0.978513,0.0


In [72]:
# rename Long_ column to long and Lat column to lat to upload to mapbox
df_covid_0424_repeat = df_covid_0424_repeat.rename(columns = {"Lat": "lat",
                                                                     "Long_": "lon"})

In [73]:
density_heatmap = px.density_mapbox(df_covid_0424_repeat, # dataframe with data for the heatmap 
                                    lat = "lat", 
                                    lon = "lon", 
                                    z = "Confirmed", 
                                    hover_data = ["Confirmed", "Deaths", "cases_per1000", "deaths_per1000"],
                                    hover_name = "Combined_Key",
                                    center = dict(lat=37.0902, 
                                                  lon= -95.7129), # center of the US
                                    zoom = 2.5, 
                                    labels = {"Confirmed": "Cumulative Confirmed Cases",
                                         "Deaths": "Cumulative Confirmed Deaths",
                                         "cases_per1000": "Cases per 1,000 People",
                                         "deaths_per1000": "Deaths per 1,000 People"
                                        }, 
                                    color_continuous_scale= "Inferno",
                                    title = "Density Heatmap of Confirmed COVID-19 Cases in the US",
                                    radius = 25,
                                   )

In [74]:
# instead, we can view if we export as an html
density_heatmap.write_html("density_heatmap_covid19_april24.html")

In [75]:
# save geojson file in our notebook
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    us_counties_shape = json.load(response)

In [76]:
# upload the nyt covid-19 data
covid_nyt_data = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

In [77]:
# make data into a dataframe
df_county_nyt = pd.read_csv(covid_nyt_data)

In [78]:
# preview data
df_county_nyt.tail()

,date,county,state,fips,cases,deaths
89767,2020-04-25,Sublette,Wyoming,56035.0,1,0
89768,2020-04-25,Sweetwater,Wyoming,56037.0,10,0
89769,2020-04-25,Teton,Wyoming,56039.0,64,1
89770,2020-04-25,Uinta,Wyoming,56041.0,6,0
89771,2020-04-25,Washakie,Wyoming,56043.0,4,0


In [79]:
# convert fips to an integer then string then forward fill 0s to get 5 digits

#remove null values
df_county_nyt = df_county_nyt[df_county_nyt["fips"].notnull()]
# edit datatypes
df_county_nyt["fips"] = df_county_nyt["fips"].astype(int)
df_county_nyt["fips"] = df_county_nyt["fips"].astype(str)
df_county_nyt["fips"] = df_county_nyt["fips"].str.zfill(5)

In [80]:
df_county_nyt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88675 entries, 0 to 89771
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    88675 non-null  object
 1   county  88675 non-null  object
 2   state   88675 non-null  object
 3   fips    88675 non-null  object
 4   cases   88675 non-null  int64 
 5   deaths  88675 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 4.7+ MB


In [81]:
# convert date column to datetime
df_county_nyt["date"] = pd.to_datetime(df_county_nyt["date"])

In [82]:
# make column with month name
df_county_nyt["month_name"] = df_county_nyt["date"].dt.month_name()

In [83]:
# make column with month day
df_county_nyt["day_of_month"] = df_county_nyt["date"].dt.day

In [84]:
df_county_nyt.head()

,date,county,state,fips,cases,deaths,month_name,day_of_month
0,2020-01-21,Snohomish,Washington,53061,1,0,January,21
1,2020-01-22,Snohomish,Washington,53061,1,0,January,22
2,2020-01-23,Snohomish,Washington,53061,1,0,January,23
3,2020-01-24,Cook,Illinois,17031,1,0,January,24
4,2020-01-24,Snohomish,Washington,53061,1,0,January,24


In [85]:
# combine month and day into a string
df_county_nyt["month_day_name"] = df_county_nyt["month_name"] + " " + df_county_nyt["day_of_month"].astype(str)

In [86]:
# preview data
df_county_nyt.head()

,date,county,state,fips,cases,deaths,month_name,day_of_month,month_day_name
0,2020-01-21,Snohomish,Washington,53061,1,0,January,21,January 21
1,2020-01-22,Snohomish,Washington,53061,1,0,January,22,January 22
2,2020-01-23,Snohomish,Washington,53061,1,0,January,23,January 23
3,2020-01-24,Cook,Illinois,17031,1,0,January,24,January 24
4,2020-01-24,Snohomish,Washington,53061,1,0,January,24,January 24


In [87]:
# merge 2019 population data onto nyt covid-19 df
df_county_nyt = pd.merge(df_county_nyt,
                         df_countypop[["FIPS","POPESTIMATE2019"]],
                         left_on = "fips",
                         right_on = "FIPS",
                         how = "left")

In [88]:
# make columns to calculate number of cases and deaths per 100000 residents
# round the values to the 3rd decimal

df_county_nyt["cases_per100000"] = round(((df_county_nyt["cases"]/df_county_nyt["POPESTIMATE2019"])*100000),3)
df_county_nyt["deaths_per100000"] = round(((df_county_nyt["deaths"]/df_county_nyt["POPESTIMATE2019"])*100000),3)

In [89]:
df_county_nyt.head()

,date,county,state,fips,cases,deaths,month_name,day_of_month,month_day_name,FIPS,POPESTIMATE2019,cases_per100000,deaths_per100000
0,2020-01-21,Snohomish,Washington,53061,1,0,January,21,January 21,53061,822083,0.122,0.0
1,2020-01-22,Snohomish,Washington,53061,1,0,January,22,January 22,53061,822083,0.122,0.0
2,2020-01-23,Snohomish,Washington,53061,1,0,January,23,January 23,53061,822083,0.122,0.0
3,2020-01-24,Cook,Illinois,17031,1,0,January,24,January 24,17031,5150233,0.019,0.0
4,2020-01-24,Snohomish,Washington,53061,1,0,January,24,January 24,53061,822083,0.122,0.0


In [90]:
print(px.colors.sequential.Plasma)

['#0d0887', '#46039f', '#7201a8', '#9c179e', '#bd3786', '#d8576b', '#ed7953', '#fb9f3a', '#fdca26', '#f0f921']


In [91]:
# make a logai=rithmic color scale
covid_colorscale = [
        [0, '#0d0887'],       #0 cases
        [1./1000, '#7201a8'], #10 cases
        [1./100, '#bd3786'],  #100 cases
        [1./10, '#ed7953'],   #1000 cases
        [1., '#fdca26'],      #10000 cases
]

In [92]:
# make a map of covid cases on april 14, 2020
covid_uscounty_color = px.choropleth(df_county_nyt, # dataframe with data for choropleth
                                     geojson=us_counties_shape, # shape, geospatial data geojson
                                     locations='fips', # column in geojson that denotes the shapes
                                     color='cases_per100000', # column in df that denotes the color scale
                                     animation_group = "month_day_name",
                                     animation_frame = "month_day_name",
                                     hover_name = df_county_nyt["county"] + ", " + df_county_nyt["state"],
                                     hover_data = ["cases", "deaths", "cases_per100000", "deaths_per100000"],
                                     color_continuous_scale=covid_colorscale, # custom color scale to better show exponential growth
                                     scope="usa", # scope of map (world, USA, or any continent)
                                     labels={'cases':'Cummulative Reported COVID-19 Cases',
                                             "deaths": "Cummulative Reported COVID-19 Deaths",
                                             "month_day_name": "Date",
                                             "fips": "FIPS Code",
                                            "cases_per100000": "Cummulative Reported Cases per 100,000 People",
                                             "deaths_per100000": "Cummulative Reported Deaths per 100,000 People"
                                            }, # renaming any columns used
                                     title = "Spread of COVID-19 Cases in US Counties Per Capita<br>January 21, 2020-April 15, 2020"
                                          
                          )

In [94]:
# export animation to html
covid_uscounty_color.write_html("covid19_cases_us_county.html")

In [106]:
df_county_nyt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88675 entries, 0 to 88674
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              88675 non-null  datetime64[ns]
 1   county            88675 non-null  object        
 2   state             88675 non-null  object        
 3   fips              88675 non-null  object        
 4   cases             88675 non-null  int64         
 5   deaths            88675 non-null  int64         
 6   month_name        88675 non-null  object        
 7   day_of_month      88675 non-null  int64         
 8   month_day_name    88675 non-null  object        
 9   FIPS              88675 non-null  object        
 10  POPESTIMATE2019   88675 non-null  int64         
 11  cases_per100000   88675 non-null  float64       
 12  deaths_per100000  88675 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(6)
memory usage: 9.5+ MB


In [129]:
# sort values by cases
df_total_cases = df_county_nyt.sort_values(by = "cases", ascending = False)

In [130]:
# top 20
df_total_cases_top20 = df_total_cases.head(20)

In [131]:
df_total_cases.head(20)

,date,county,state,fips,cases,deaths,month_name,day_of_month,month_day_name,FIPS,POPESTIMATE2019,cases_per100000,deaths_per100000,county_state_name
87532,2020-04-25,Nassau,New York,36059,33798,1917,April,25,April 25,36059,1356924,2490.781,141.275,Nassau New York
84743,2020-04-24,Nassau,New York,36059,32765,1867,April,24,April 24,36059,1356924,2414.653,137.591,Nassau New York
81965,2020-04-23,Nassau,New York,36059,32124,1813,April,23,April 23,36059,1356924,2367.413,133.611,Nassau New York
79192,2020-04-22,Nassau,New York,36059,31555,1764,April,22,April 22,36059,1356924,2325.480,130.000,Nassau New York
87551,2020-04-25,Suffolk,New York,36103,31368,1085,April,25,April 25,36103,1476601,2124.338,73.480,Suffolk New York
76432,2020-04-21,Nassau,New York,36059,31079,1717,April,21,April 21,36059,1356924,2290.401,126.536,Nassau New York
73679,2020-04-20,Nassau,New York,36059,30677,1638,April,20,April 20,36059,1356924,2260.775,120.714,Nassau New York
84762,2020-04-24,Suffolk,New York,36103,30606,1035,April,24,April 24,36103,1476601,2072.733,70.093,Suffolk New York
70932,2020-04-19,Nassau,New York,36059,30013,1577,April,19,April 19,36059,1356924,2211.841,116.219,Nassau New York
81984,2020-04-23,Suffolk,New York,36103,29567,994,April,23,April 23,36103,1476601,2002.369,67.317,Suffolk New York


In [133]:
# make a graph to show percent of largest population 18 and under
bar_total_cases_top20 = px.bar(df_total_cases_top20,
                             x = "county",
                             y = "cases",
                             title = "US Counties with the Highest Total Cases in a day",
                             labels = {"county": "County", "cases": "Total Cases in a Day"})

In [134]:
bar_total_cases_top20